In [1]:
%cd ..

/home/jupyter-tim/ba-tim


In [2]:
from drlqap.taskgenerators import generators
import scipy.optimize
from drlqap.simplesolver import solve_qap_backtracking, solve_qap_maxgreedy, solve_qap_faq, solve_random
from drlqap.gurobi import solve_qap_gurobi
import numpy as np
from drlqap.qap import QAP
from drlqap.qaplib import load_qap
from drlqap.evaltools import load_checkpoints
from pathlib import Path

In [3]:
mini_agents = load_checkpoints(Path('runs/dqn_dense_ms_ec_eps0_mini/lr5e-4_s2/'))

runs/dqn_dense_ms_ec_eps0_mini/lr5e-4_s2/checkpoint_0.pth
runs/dqn_dense_ms_ec_eps0_mini/lr5e-4_s2/checkpoint_1000.pth
runs/dqn_dense_ms_ec_eps0_mini/lr5e-4_s2/checkpoint_2000.pth
runs/dqn_dense_ms_ec_eps0_mini/lr5e-4_s2/checkpoint_3000.pth
runs/dqn_dense_ms_ec_eps0_mini/lr5e-4_s2/checkpoint_4000.pth
runs/dqn_dense_ms_ec_eps0_mini/lr5e-4_s2/checkpoint_end.pth


In [4]:
a2c_agents = load_checkpoints(Path('runs/a2c_ms100x_mediumrandoms/lr4e-5_s3'))

runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_0.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_1000.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_2000.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_3000.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_4000.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_5000.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_6000.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_7000.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_8000.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_9000.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_10000.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_11000.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_12000.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_13000.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_14000.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_15000.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_

In [89]:
a2c_longer_agents = load_checkpoints(Path('runs/a2c_ms100x_mediumrandoms/lr2e-5_s1'))

runs/a2c_ms100x_mediumrandoms/lr2e-5_s1/checkpoint_0.pth
runs/a2c_ms100x_mediumrandoms/lr2e-5_s1/checkpoint_1000.pth
runs/a2c_ms100x_mediumrandoms/lr2e-5_s1/checkpoint_2000.pth
runs/a2c_ms100x_mediumrandoms/lr2e-5_s1/checkpoint_3000.pth
runs/a2c_ms100x_mediumrandoms/lr2e-5_s1/checkpoint_4000.pth
runs/a2c_ms100x_mediumrandoms/lr2e-5_s1/checkpoint_5000.pth
runs/a2c_ms100x_mediumrandoms/lr2e-5_s1/checkpoint_6000.pth
runs/a2c_ms100x_mediumrandoms/lr2e-5_s1/checkpoint_7000.pth
runs/a2c_ms100x_mediumrandoms/lr2e-5_s1/checkpoint_8000.pth
runs/a2c_ms100x_mediumrandoms/lr2e-5_s1/checkpoint_9000.pth
runs/a2c_ms100x_mediumrandoms/lr2e-5_s1/checkpoint_10000.pth
runs/a2c_ms100x_mediumrandoms/lr2e-5_s1/checkpoint_11000.pth
runs/a2c_ms100x_mediumrandoms/lr2e-5_s1/checkpoint_12000.pth
runs/a2c_ms100x_mediumrandoms/lr2e-5_s1/checkpoint_13000.pth
runs/a2c_ms100x_mediumrandoms/lr2e-5_s1/checkpoint_14000.pth
runs/a2c_ms100x_mediumrandoms/lr2e-5_s1/checkpoint_15000.pth
runs/a2c_ms100x_mediumrandoms/lr2e-5_

In [ ]:
dqn_small = load_checkpoints(Path('runs/a2c_ms100x_mediumrandoms/lr2e-5_s1'))

In [5]:
qap = load_qap('qapdata/bur26c.dat')
mini_agents[-1].solve(qap)

(5995219.0,
 [20,
  25,
  16,
  13,
  0,
  3,
  6,
  21,
  15,
  2,
  7,
  10,
  23,
  9,
  22,
  12,
  18,
  17,
  19,
  14,
  11,
  5,
  4,
  8,
  1,
  24])

In [6]:
def evaluate_set(solver, problem):
    values = []
    test_set = generators[problem].test_set()
    for qap in test_set:
        v_solver, assignment = solver(qap)
        v = qap.compute_value(assignment)
        if not np.isclose(v, v_solver):
            print (f"solver outputs incorrect value (got {v_solver}, actual {v})")
        values.append(v)
    return np.mean(values)

In [190]:
def evaluate_comparison(solvers, problem, cached_results=None):
    results = cached_results or {}
    for name, solver in solvers.items():
        if name not in results:
            print(f"Evaluating {name}...")
            values = {}
        else:
            values = results[name]
        test_set = generators[problem].test_set()
        for qap in test_set:
            if qap.name not in values:
                print(f"Evaluating {name} on {qap.name}...")
                v_solver, assignment = solver(qap)
                v = qap.compute_value(assignment)
                if not np.isclose(v, v_solver):
                    print (f"solver {name} outputs incorrect value on {qap.name} (got {v_solver}, actual {v})")
                values[qap.name] = float(v)
        results[name] = values
    return results
    

In [191]:
solvers = {
    'faq': solve_qap_faq,
    'a2c': a2c_agents[-1].solve,
    'dqn_mini': mini_agents[-1].solve,
    'random': solve_random,
    'a2c_longer': a2c_longer_agents[-1].solve,
    'max_greedy': solve_qap_maxgreedy,
    'dqn_small': load_checkpoints(Path('runs/dqn_dense_ms_ec_eps0_smallrandoms/lr5e-4_s1'))[-1].solve,
    'dqn_medium': load_checkpoints(Path('runs/dqn_dense_ms_ec_eps0_mediumrandoms/lr5e-4_s1'))[-1].solve,
    'reinforce': load_checkpoints(Path('runs/reinforce_ms100x_smallrandoms/lr1e-4_s4/'))[-1].solve,
}

runs/dqn_dense_ms_ec_eps0_smallrandoms/lr5e-4_s1/checkpoint_0.pth
runs/dqn_dense_ms_ec_eps0_smallrandoms/lr5e-4_s1/checkpoint_1000.pth
runs/dqn_dense_ms_ec_eps0_smallrandoms/lr5e-4_s1/checkpoint_2000.pth
runs/dqn_dense_ms_ec_eps0_smallrandoms/lr5e-4_s1/checkpoint_3000.pth
runs/dqn_dense_ms_ec_eps0_smallrandoms/lr5e-4_s1/checkpoint_4000.pth
runs/dqn_dense_ms_ec_eps0_smallrandoms/lr5e-4_s1/checkpoint_5000.pth
runs/dqn_dense_ms_ec_eps0_smallrandoms/lr5e-4_s1/checkpoint_6000.pth
runs/dqn_dense_ms_ec_eps0_smallrandoms/lr5e-4_s1/checkpoint_7000.pth
runs/dqn_dense_ms_ec_eps0_smallrandoms/lr5e-4_s1/checkpoint_8000.pth
runs/dqn_dense_ms_ec_eps0_smallrandoms/lr5e-4_s1/checkpoint_9000.pth
runs/dqn_dense_ms_ec_eps0_smallrandoms/lr5e-4_s1/checkpoint_end.pth
runs/dqn_dense_ms_ec_eps0_mediumrandoms/lr5e-4_s1/checkpoint_0.pth
runs/dqn_dense_ms_ec_eps0_mediumrandoms/lr5e-4_s1/checkpoint_1000.pth
runs/dqn_dense_ms_ec_eps0_mediumrandoms/lr5e-4_s1/checkpoint_2000.pth
runs/dqn_dense_ms_ec_eps0_mediumrandom

In [194]:
qaplib_results = evaluate_comparison(solvers, 'qaplib_all_100_normalized', cached_results=qaplib_results)

Evaluating a2c on sko100c...
Evaluating a2c on sko100d...
Evaluating a2c on sko100e...
Evaluating a2c on sko100f...
Evaluating a2c on sko72...
Evaluating a2c on sko81...
Evaluating a2c on sko90...
Evaluating a2c on tai100a...
Evaluating a2c on tai100b...
Evaluating a2c on tai80a...
Evaluating a2c on tai80b...
Evaluating a2c on wil100...
Evaluating dqn_mini on lipa70a...
Evaluating dqn_mini on lipa70b...
Evaluating dqn_mini on lipa80a...
Evaluating dqn_mini on lipa80b...
Evaluating dqn_mini on lipa90a...
Evaluating dqn_mini on lipa90b...
Evaluating dqn_mini on sko100a...
Evaluating dqn_mini on sko100b...
Evaluating dqn_mini on sko100c...
Evaluating dqn_mini on sko100d...
Evaluating dqn_mini on sko100e...
Evaluating dqn_mini on sko100f...
Evaluating dqn_mini on sko72...
Evaluating dqn_mini on sko81...
Evaluating dqn_mini on sko90...
Evaluating dqn_mini on tai100a...
Evaluating dqn_mini on tai100b...
Evaluating dqn_mini on tai80a...
Evaluating dqn_mini on tai80b...
Evaluating dqn_mini on 

In [195]:
ngm_g5k_results = {
"bur26a": 5621774, 
"bur26b": 3927943, 
"bur26c": 5608065, 
"bur26d": 3962317, 
"bur26e": 5536142, 
"bur26f": 3949711, 
"bur26g": 10433439, 
"bur26h": 7348866, 
"chr12a": 14940, 
"chr12b": 14984, 
"chr12c": 16346, 
"chr15a": 20442, 
"chr15b": 22048, 
"chr15c": 24190, 
"chr18a": 33124, 
"chr18b": 2504, 
"chr20a": 5178, 
"chr20b": 5766, 
"chr20c": 49770, 
"chr22a": 9348, 
"chr22b": 9006, 
"chr25a": 11648, 
"els19": 27029748, 
"esc16a": 78, 
"esc16b": 292, 
"esc16c": 174, 
"esc16d": 20, 
"esc16e": 32, 
"esc16f": 0, 
"esc16g": 32, 
"esc16h": 1004, 
"esc16i": 18, 
"esc16j": 8, 
"esc32a": 298, 
"esc32b": 368, 
"esc32c": 754, 
"esc32d": 284, 
"esc32e": 2, 
"esc32g": 10, 
"esc32h": 534, 
"esc64a": 200, 
"esc128": 242, 
"had12": 1700, 
"had14": 2866, 
"had16": 3902, 
"had18": 5558, 
"had20": 7300, 
"kra30a": 114410, 
"kra30b": 118130, 
"kra32": 120930, 
"lipa20a": 3853, 
"lipa20b": 33125, 
"lipa30a": 13631, 
"lipa30b": 187607, 
"lipa40a": 32454, 
"lipa40b": 601848, 
"lipa50a": 63671, 
"lipa50b": 1523856, 
"lipa60a": 109595, 
"lipa60b": 3208501, 
"lipa70a": 173220, 
"lipa70b": 5890161, 
"lipa80a": 257663, 
"lipa80b": 9983040, 
"lipa90a": 366508, 
"lipa90b": 16076956, 
"nug12": 634, 
"nug14": 1156, 
"nug15": 1318, 
"nug16a": 1836, 
"nug16b": 1396, 
"nug17": 1980, 
"nug18": 2242, 
"nug20": 2936, 
"nug21": 2916, 
"nug22": 4298, 
"nug24": 4234, 
"nug25": 4420, 
"nug27": 6208, 
"nug28": 6128, 
"nug30": 7294, 
"rou12": 264898, 
"rou15": 403872, 
"rou20": 817776, 
"scr12": 36292, 
"scr15": 68768, 
"scr20": 154636, 
"sko42": 18716, 
"sko49": 27554, 
"sko56": 40684, 
"sko64": 56222, 
"sko72": 76870, 
"sko81": 104710, 
"sko90": 132942, 
"sko100a": 172810, 
"sko100b": 175588, 
"sko100c": 169806, 
"sko100d": 170816, 
"sko100e": 170958, 
"sko100f": 169986, 
"ste36a": 16768, 
"ste36b": 43248, 
"ste36c": 12988352, 
"tai12a": 255158, 
"tai12b": 47252044, 
"tai15a": 436968, 
"tai15b": 52871608, 
"tai17a": 544754, 
"tai20a": 806382, 
"tai20b": 140704160, 
"tai25a": 1352912, 
"tai25b": 518647040, 
"tai30a": 2065706, 
"tai30b": 896379008, 
"tai35a": 2786748, 
"tai35b": 377687744, 
"tai40a": 3610604, 
"tai40b": 917498816, 
"tai50a": 5677282, 
"tai50b": 614638528, 
"tai60a": 8281996, 
"tai60b": 862969152, 
"tai64c": 2133738, 
"tai80a": 15283138, 
"tai80b": 1120577408, 
"tai100a": 23644528, 
"tai100b": 1612020992, 
"tai150b": 628349568, 
"tho30": 185622, 
"tho40": 304878, 
"tho150": 9557766, 
"wil50": 53418, 
"wil100": 294172,
}

qaplib_results["ngm_g5k"] = ngm_g5k_results

In [196]:
from collections import defaultdict

def results_by_qap(results_by_solver):
    out = defaultdict(dict)
    for solver, results in results_by_solver.items():
        for qap, value in results.items():
            out[qap][solver] = value
    return dict(out)

In [202]:
benchmark = 'faq'
compare = ['a2c', 'reinforce', 'ngm_g5k', 'dqn_medium']
count_by_solver = defaultdict(lambda: 0)
for qap, results in results_by_qap(qaplib_results).items():
    # Skip qaps where one of the solvers has no result
    incomplete_results = False
    for solver in compare:
        if solver not in results:
            print(solver, "has no data on", qap)
            incomplete_results = True
    if incomplete_results:
        print("Skipping", qap)
        continue

    ordered_results = sorted([s for s in results if s in compare], key=lambda s: results[s])
    base = results[benchmark]
    gap_to_benchmark = [f"{solver:>10}: {(results[solver]/base-1)*100:>6.4}" for solver in ordered_results]
    print(f'{f"[{qap}]":>11} ref: {base:>11.10} | {", ".join(gap_to_benchmark)}')
    count_by_solver[ordered_results[0]] += 1

print("\nNumber of times best value achieved")
for solver, count in count_by_solver.items():
    print(solver, count)

   [bur26a] ref:   5434914.0 |  reinforce:  2.425,    ngm_g5k:  3.438, dqn_medium:  5.047,        a2c:  10.37
   [bur26b] ref:   3830246.0 |    ngm_g5k:  2.551,  reinforce:  3.513, dqn_medium:  5.829,        a2c:  12.82
   [bur26c] ref:   5434782.0 |  reinforce:  2.378, dqn_medium:  2.483,    ngm_g5k:  3.188,        a2c:  13.22
   [bur26d] ref:   3826287.0 |    ngm_g5k:  3.555, dqn_medium:  3.773,  reinforce:  4.625,        a2c:  15.65
   [bur26e] ref:   5399558.0 |    ngm_g5k:   2.53,  reinforce:  3.182, dqn_medium:  6.652,        a2c:  16.07
   [bur26f] ref:   3784562.0 |  reinforce:  3.731,    ngm_g5k:  4.364, dqn_medium:  6.518,        a2c:  18.33
   [bur26g] ref:  10145072.0 |    ngm_g5k:  2.842,  reinforce:  3.068, dqn_medium:  6.373,        a2c:  13.64
   [bur26h] ref:   7119460.0 |    ngm_g5k:  3.222, dqn_medium:  5.759,  reinforce:  7.228,        a2c:  15.29
   [chr12a] ref:     33082.0 |    ngm_g5k: -54.84,        a2c: -54.49, dqn_medium:  5.592,  reinforce:  16.98
   [chr12b

In [166]:
qaplib_results

{'faq': {'bur26a': 5434914.0,
  'bur26b': 3830246.0,
  'bur26c': 5434782.0,
  'bur26d': 3826287.0,
  'bur26e': 5399558.0,
  'bur26f': 3784562.0,
  'bur26g': 10145072.0,
  'bur26h': 7119460.0,
  'chr12a': 33082.0,
  'chr12b': 10468.0,
  'chr12c': 13088.0,
  'chr15a': 19852.0,
  'chr15b': 9112.0,
  'chr15c': 16884.0,
  'chr18a': 15440.0,
  'chr18b': 1792.0,
  'chr20a': 3172.0,
  'chr20b': 3206.0,
  'chr20c': 19836.0,
  'chr22a': 8430.0,
  'chr22b': 8714.0,
  'chr25a': 5580.0,
  'els19': 21297242.0,
  'esc16a': 70.0,
  'esc16b': 320.0,
  'esc16c': 168.0,
  'esc16d': 62.0,
  'esc16e': 30.0,
  'esc16g': 30.0,
  'esc16h': 1518.0,
  'esc16i': 14.0,
  'esc16j': 8.0,
  'esc32a': 160.0,
  'esc32b': 196.0,
  'esc32c': 650.0,
  'esc32d': 226.0,
  'esc32e': 2.0,
  'esc32g': 10.0,
  'esc32h': 486.0,
  'esc64a': 118.0,
  'had12': 1666.0,
  'had14': 2726.0,
  'had16': 3736.0,
  'had18': 5442.0,
  'had20': 6988.0,
  'kra30a': 96620.0,
  'kra30b': 94810.0,
  'kra32': 92930.0,
  'lipa20a': 3772.0,
  'lip